In [376]:
import pandas as pd
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem

import ripser
import teaspoon.TDA.Draw as Draw
import persim

from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR

from tqdm import trange, tqdm

In [2]:
dfs = pd.read_excel('1400.xlsx')
dfb = pd.read_excel('35000.xlsx')

In [7]:
dfs

,Title,"IC50, mmg/ml","CC50-MDCK, mmg/ml",SI,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,SMILES,Pictures
0,1007-Ya-213,2.7,500.0,185.185185,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,50.0
1,1007-Ya-213,0.7,447.0,638.571429,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,51.0
2,1008-Ya-187,9.9,144.0,14.545455,250.431,1,0,15.60,CCN(CC)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,52.0
3,1009-As-106,8.3,500.0,60.240964,222.377,1,0,15.60,CN(C)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,53.0
4,1010-Ya-208,39.4,143.0,3.629442,239.361,2,0,29.54,CN(C)CC(=O)O[C@H]1C[C@H](CC2)C(C)(C)[C@@]12C,54.0
...,...,...,...,...,...,...,...,...,...,...
1463,CHEMBL4251117,7.3,530.6,72.800000,238.330,3,0,39.44,CCCCCCCc1c(OC)c(C)c(=O)oc1,NaN
1464,CHEMBL4217515,3.8,4733.9,1250.000000,427.417,2,1,124.74,COc(cc1)cc(c12)ccc(c2)-c(c3C([O-])=O)c(C([O-])...,NaN
1465,CHEMBL4205814,0.3,4733.9,14285.700000,427.417,2,1,124.74,COc(cc1)cc(c12)ccc(c2)-c(c3C([O-])=O)c(C([O-])...,NaN
1466,CHEMBL4212064,0.7,3662.8,5555.600000,320.304,1,0,89.49,c1ccc(C([O-])=O)c(c1C([O-])=O)-c(c2)ccc(c23)cc...,NaN


In [255]:
bar = trange(dfs.shape[0])

def represent(smiles, maxdim=2):
    m = Chem.MolFromSmiles(smiles)
    m = Chem.AddHs(m)
    AllChem.EmbedMolecule(m)
    r = pd.Series({'SMILES': smiles, 'dmgs': None})
    try:
        AllChem.MMFFOptimizeMolecule(m)
        coords = m.GetConformer().GetPositions()
        # r['ma'] = np.mean([a.GetAtomicNum() for a in Chem.RemoveAllHs(m).GetAtoms()])
        r['dmgs'] = ripser.ripser(coords, maxdim)['dgms']
    except:
        pass
    bar.update()
    return r

  0%|          | 0/1468 [00:01<?, ?it/s]


In [256]:
tdfs = dfs.SMILES.apply(represent)

 50%|█████     | 739/1468 [06:02<08:57,  1.36it/s][15:51:13] Conflicting single bond directions around double bond at index 55.
[15:51:13]   BondStereo set to STEREONONE and single bond directions set to NONE.
100%|██████████| 1468/1468 [09:00<00:00, 14.11it/s]

In [321]:
# Приведение всех диаграмм первого порядка к одному размеру l - с ним можно еще поиграться
def clippad(x, l=10):
    x = x[1][:l]
    x = np.pad(x, [(0, l-x.shape[0]), (0, 0)])
    return pd.Series(x.flatten())
X = tdfs['dmgs'].dropna().apply(clippad)

In [294]:
Y = dfs['IC50, mmg/ml'].loc[X.index]
scaler = StandardScaler().fit(Y[Y<310].values.reshape(-1,1))

In [350]:
scores=cross_val_score(
    CatBoostRegressor(verbose=0),
    X[Y<310], scaler.transform(Y[Y<310].values[:,None]),
    cv=ShuffleSplit(5, train_size=844/X.shape[0], random_state=42),
    scoring=make_scorer(mean_squared_error)
)
print(*scores)
print(scores.mean())

0.8344886550595321 0.998750867898553 1.016297434962489 0.9577063663021139 0.9806719614495499
0.9575830571344476


In [352]:
scores=cross_val_score(
    CatBoostRegressor(verbose=0),
    X[Y<310], Y[Y<310],
    cv=ShuffleSplit(5, train_size=844/X.shape[0], random_state=42)
)
print(*scores)
print(scores.mean())

0.1557658673996163 0.05769518946191454 0.09838750385780948 0.06600560704550507 0.018357186328586073
0.07924227081868629


In [377]:
# метрика близости персистентных диаграмм первого порядка
def dist(a, b): return persim.bottleneck(a.reshape((-1,2)), b.reshape((-1,2)))

scores=cross_val_score(
    SVR(),
    X[Y<310], Y[Y<310],
    cv=ShuffleSplit(5, train_size=844/X.shape[0], random_state=42)
)
print(*scores)
print(scores.mean())

-0.1825341858527021 -0.1935171214538598 -0.21062378261396453 -0.21746362779244421 -0.1541995423359408
-0.19166765200978228
